In [3]:
#import libraries
import gradio as gr
import numpy
import sys
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import csv
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics
import matplotlib.pyplot as plt
import gradio as gr

In [5]:
# Read a CSV file and determine X features & y target
df=pd.read_csv("C:\\project\\datatest.csv")
X=df.drop("level",axis=1).values
y=df.level.values


#Build a model & get a Scores
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X, y)

k_folds = KFold(n_splits = 5)
scores = cross_val_score(knn, X, y, cv = k_folds)
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))
y_true=df
expected=y_true
predict = knn.predict(X)
confusion_matrix(y_true['level'], y_pred=predict)
print(metrics.classification_report(y_true['level'], y_pred=predict))


Cross Validation Scores:  [0.81818182 0.71875    0.96875    0.96875    1.        ]
Average CV Score:  0.8948863636363636
Number of CV Scores used in Average:  5


In [6]:
#Define function for prediction(classify) features with knn, then define a dictionary for grades of output
def predict_grade(ppm, SR, repeat_of_alarm, not_po_ka_yoke, high_price):
    """
    Predicts ppm, SR, repeat_of_alarm, not_po_ka_yoke, high_price.
        
    Returns:
        Level : A, B, C, D, E, F".
    """

    # Reshape the input data for prediction
    input_data = np.array([ppm, SR, repeat_of_alarm, not_po_ka_yoke, high_price]).reshape(1, -1)
    
    # Get the prediction
    prediction = knn.predict(input_data)
    
    # Return the result
    if prediction[0] == 0:
        return "A"
    elif prediction[0] == 1:
        return "B"
    elif prediction[0] == 2:
        return "C"
    elif prediction[0] == 3:
        return "D"
    elif prediction[0] == 4:
        return "E"
    elif prediction[0] == 5:
        return "F"
    
    else:
        raise Exception

In [7]:
# Set the minimum, maximum, and default values for the sliders
# This is optional
sl_min = int(X[:,0].min().round(2))
sl_max = int(X[:,0].max().round(2))
sl_default = int(X[:,0].mean().round(2))

sw_min = int(X[:,1].min().round(2))
sw_max = int(X[:,1].max().round(2))
sw_default = int(X[:,1].mean().round(2))



sb_min = int(X[:,2].min().round(2)) 
sb_max = int(X[:,2].max().round(2)) 
sb_default = int(X[:,2].mean().round(2))




sz_min =int(X[:,3].min().round(2)) 
sz_max = int(X[:,3].max().round(2))
sz_default = int(X[:,3].mean().round(2))




sh_min = int(X[:,4].min().round(2))
sh_max = int(X[:,4].max().round(2)) 
sh_default = int(X[:,4].mean().round(2))

In [8]:
#Create the input component for Gradio since we are expecting 5 inputs amd 1 output as Grade
iface = gr.Interface(
    fn=predict_grade,
    inputs=[
        gr.components.Slider(minimum=sl_min, maximum=sl_max, 
        value=sl_default, label="PPM"),
        gr.components.Slider(minimum=sw_min, maximum=sw_max, 
        value=sw_default, label="SR"),
        gr.components.Slider(minimum=sb_min, maximum=sb_max, 
        value=sb_default, label="Repeat of Alarm"),
        gr.components.Slider(minimum=sz_min, maximum=sz_max, 
        value=sz_default, label="Not Pokayoka"),
        gr.components.Slider(minimum=sh_min, maximum=sh_max, 
        value=sh_default, label="High price")
        
    ], 
    outputs=gr.components.Textbox(label="Grade of Part"),
    title="Classifier",
    description="""Enter 0 for NOT & 1 for YES""")

iface.launch(share=True,debug=True)

Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


Keyboard interruption in main thread... closing server.
